In [12]:
# Import packages and load the data
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
#import matplotlib
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import cm
#cmap1 = matplotlib.cm.get_cmap('summer')
# This next command is specifically for Jupyter Notebook
#%matplotlib notebook

df = np.loadtxt('data/MacroSeries.txt', delimiter = ",")  #columns: c, k, w, r

In [13]:
'''
---------------------------------------------------------------------
We fisrtly need to specifiy the data_moments
However, in the case of this homework, the data_moments are all zeros
---------------------------------------------------------------------

'''
moms_data = np.array([[1e-30], [1e-30], [1e-30], [1e-30]])

In [14]:
'''
---------------------------------------------------------------------
We then need to write a function to derive the model moments
---------------------------------------------------------------------

'''
def model_moments(c_vec, k_vec, w_vec, r_vec, alpha, beta, rho, mu):
    '''
    ---------------------------------------------
    Function to derive model moments
    ---------------------------------------------
    We will firstly use equation (4) to derive the series z
    Then we will derive the four model moments
    
    '''
    z_vec = np.log(r_vec) +(1 - alpha)*np.log(k_vec) - np.log(alpha)
    model_mom1 = (z_vec - rho*np.append(mu, z_vec[:-1]) - (1 - rho)*mu).mean()
    model_mom2 = ((z_vec - rho*np.append(mu, z_vec[:-1]) - (1 - rho)*mu)*np.append(mu, z_vec[:-1])).mean()
    model_mom3 = ((beta*alpha*np.exp(z_vec[1:])*(k_vec[1:]**(alpha - 1))*c_vec[:-1]/c_vec[1:]) - 1).mean()
    model_mom4 = (((beta*alpha*np.exp(z_vec[1:])*(k_vec[1:]**(alpha - 1))*c_vec[:-1]/c_vec[1:]) - 1)*w_vec[:-1]).mean()
    
    return model_mom1, model_mom2, model_mom3, model_mom4


In [15]:
def err_vec(c_vec, k_vec, w_vec, r_vec, alpha, beta, rho, mu, moms_data, simple = False):
    '''
    ---------------------------------------------------
    Function to return the error vector
    ---------------------------------------------------
    
    '''
    model_mom1, model_mom2, model_mom3, model_mom4 = model_moments(c_vec, k_vec, w_vec, r_vec, alpha, beta, rho, mu)
    moms_model = np.array([[model_mom1], [model_mom2], [model_mom3], [model_mom4]])
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data)/moms_model
    
    return err_vec

    

In [16]:
def crit(params, *args):
    alpha, beta, rho, mu = params
    c_vec, k_vec, w_vec, r_vec, moms_data, W, simple = args
    err = err_vec(c_vec, k_vec, w_vec, r_vec, alpha, beta, rho, mu, moms_data, simple)
    crit_val = np.dot(np.dot(err.T, W), err)
    
    return crit_val


In [18]:
#testing
alpha_init = 0.5
beta_init = 0.5
rho_init = 0.5
mu_init = 9.5
bnds = ((1e-5, 1-(1e-5)), (1e-5, 1-(1e-5)), ((1e-5)-1, 1-(1e-5)), (1e-5, None))
params_init = (alpha_init, beta_init, rho_init, mu_init)
gmm_args = (df[:, 0], df[:, 1], df[:, 2], df[:, 3], moms_data, np.eye(4), True)
results = opt.minimize(crit, params_init, args=(gmm_args),
                       method='L-BFGS-B', bounds=bnds)
results

      fun: array([[ 0.00276665]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  3.78590904e-02,   1.26107131e+02,   2.72455669e-06,
        -1.06756405e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 235
      nit: 28
   status: 0
  success: True
        x: array([ 0.45045377,  0.99      ,  0.50539438,  9.49692001])

After trying some different initials, I find that only the estimation of beta is robust to different initial guesses....